**Drive Mount**

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Extract Zip File**

In [26]:
import zipfile
import os

#zip path
zip_path = '/content/drive/MyDrive/archive (2).zip'
extract_dir = '/content'

if os.path.exists(zip_path):
  try:
# Create a ZipFile object
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
# Extract all contents to the specified directory
      zip_ref.extractall(extract_dir)
    print(f'Successfully extracted \'{zip_path}\' to \'{extract_dir}\'')
  except zipfile.BadZipFile:
    print(f'Error: \'{zip_path}\' is not a valid zip file.')
  except Exception as e:
    print(f'An error occurred: {e}')
else:
  print(f'Error: \'{zip_path}\' does not exist.')

Successfully extracted '/content/drive/MyDrive/archive (2).zip' to '/content'


**Import Libraries**

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob as gb
import glob
import cv2
import tensorflow as tf


**Train Path**

In [28]:
train = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

size = 224

**Setting Up Train Generator**

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1/255.0,
    preprocessing_function=None,
    validation_split=0.1
).flow_from_directory(
    train,
    batch_size=164,
    target_size=(size, size),
    subset='training',
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True
)


In [ ]:
train_generator.class_indices

**Visualize Training Images**

In [ ]:
# Let's display some images from the training set

classes = list(train_generator.class_indices.keys())
plt.figure(figsize=(20, 20))

for x_batch, y_batch in train_generator:
    # Create a grid of 4x4 images
    for i in range(0, 16):
        plt.subplot(4, 4, i + 1)
        plt.imshow(x_batch[i])
        plt.title(classes[np.where(y_batch[i] == 1)[0][0]])  # Set the corresponding title based on the label
        plt.grid(None)  # Turn off the grid on the subplot

    # Show the plot
    plt.show()
    break  # Exit after displaying the first batch

**Validation Generator**

In [32]:
valid = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0
).flow_from_directory(
    valid,
    target_size=(size, size),
    batch_size=164,
    class_mode='categorical',
    shuffle=False
)


Found 17572 images belonging to 38 classes.


**Test Generator**

In [33]:
# Define the path for test data
test = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

# Create a test data generator
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
    preprocessing_function=None
).flow_from_directory(
    test,
    batch_size=64,
    target_size=(224, 224),
    color_mode='rgb',  # Use 'rgba' or 'grayscale'
    class_mode='categorical',  # Use 'binary', 'sparse', 'categorical', or None as needed
    shuffle=False
)


Found 17572 images belonging to 38 classes.


In [34]:
# Retrieve class indices
test_generator.class_indices

{'Apple___Apple_scab': 0,
 'Apple___Black_rot': 1,
 'Apple___Cedar_apple_rust': 2,
 'Apple___healthy': 3,
 'Blueberry___healthy': 4,
 'Cherry_(including_sour)___Powdery_mildew': 5,
 'Cherry_(including_sour)___healthy': 6,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7,
 'Corn_(maize)___Common_rust_': 8,
 'Corn_(maize)___Northern_Leaf_Blight': 9,
 'Corn_(maize)___healthy': 10,
 'Grape___Black_rot': 11,
 'Grape___Esca_(Black_Measles)': 12,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13,
 'Grape___healthy': 14,
 'Orange___Haunglongbing_(Citrus_greening)': 15,
 'Peach___Bacterial_spot': 16,
 'Peach___healthy': 17,
 'Pepper,_bell___Bacterial_spot': 18,
 'Pepper,_bell___healthy': 19,
 'Potato___Early_blight': 20,
 'Potato___Late_blight': 21,
 'Potato___healthy': 22,
 'Raspberry___healthy': 23,
 'Soybean___healthy': 24,
 'Squash___Powdery_mildew': 25,
 'Strawberry___Leaf_scorch': 26,
 'Strawberry___healthy': 27,
 'Tomato___Bacterial_spot': 28,
 'Tomato___Early_blight': 29,
 'Toma

**Build CNN Model**

In [35]:
from tensorflow import keras

model = keras.models.Sequential()  # To build the model

model.add(keras.layers.Conv2D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu", name="Conv1", input_shape=(224, 224, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=2, name="Pool1"))

model.add(keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding="same", activation="relu", name="Conv2"))
model.add(keras.layers.MaxPooling2D(pool_size=2, name="Pool2"))

model.add(keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu", name="Conv3"))
model.add(keras.layers.MaxPooling2D(pool_size=2, name="Pool3"))

model.add(keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, padding="same", activation="relu", name="Conv4"))

model.add(keras.layers.Flatten(name="Flatten"))  # Flatten layer to convert into 1D vector
model.add(keras.layers.Dense(128, activation="relu", name="Dense1"))  # Hidden layer
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation="relu", name="Dense2"))  # Hidden layer
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(38, activation="softmax", name="Output"))  # Output layer
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv1 (Conv2D)                  │ (None, 224, 224, 32)   │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool1 (MaxPooling2D)            │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (Conv2D)                  │ (None, 112, 112, 64)   │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool2 (MaxPooling2D)            │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv3 (Conv2D)                  │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool3 (MaxPooling2D)            │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv4 (Conv2D)                  │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten (Flatten)               │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense1 (Dense)                  │ (None, 128)            │    25,690,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense2 (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 38)             │         2,470 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,125,990 (99.66 MB)

 Trainable params: 26,125,990 (99.66 MB)

 Non-trainable params: 0 (0.00 B)

None
